In [4]:
import pandas as pd

itami_wifi = pd.read_csv("../data/282073publicwirelesslan202204.csv")
itami_wifi.iloc[:5, 3:10]

,名称,名称_カナ,名称_英語,住所,方書,緯度,経度
0,有岡城跡公園,アリオカジョウセキコウエン,Arioka Castle Ruins,兵庫県伊丹市伊丹1-16,NaN,34.780851,135.421042
1,市バス総合案内所,シバスソウゴウアンナイジョ,City Bus Terminal,兵庫県伊丹市西台1-1-24,NaN,34.780171,135.413086
2,市立伊丹ミュージアム,シリツイタミミュージアム,Itami City Museum,兵庫県伊丹市宮ノ前2-5,NaN,34.781996,135.417263
3,市立伊丹ミュージアム,シリツイタミミュージアム,Itami City Museum,兵庫県伊丹市宮ノ前2-5-20,NaN,34.782622,135.417333
4,市立伊丹ミュージアム,シリツイタミミュージアム,Itami City Museum,兵庫県伊丹市宮ノ前2-5-32,NaN,34.782120,135.416952


In [6]:
itami_center = itami_wifi.loc[:, ["緯度", "経度"]].mean()
itami_center

緯度     34.781261
経度    135.422682
dtype: float64

In [13]:
import folium

itami_wifi_map = folium.Map(
    location=itami_center,
    zoom_start=15,
    width=800,
    height=600,
)
itami_wifi_map

In [16]:
marker = folium.Marker(location=itami_wifi.loc[0, ["緯度", "経度"]])
marker.add_to(itami_wifi_map)
itami_wifi_map

c:\Python312\Lib\site-packages\folium\utilities.py:93: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coords = (location[0], location[1])


In [19]:
itami_wifi.loc[:, ["緯度", "経度"]].apply(
    lambda x: folium.Marker(location=x).add_to(itami_wifi_map),
    axis=1,
)
itami_wifi_map

c:\Python312\Lib\site-packages\folium\utilities.py:93: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coords = (location[0], location[1])


In [22]:
import json
from io import StringIO
from urllib import parse, request
import folium
import geopandas as gpd

place = "区役所"
url = parse.urlunparse(
    (
        "https",
        "msearch.gsi.go.jp",
        "/address-search/AddressSearch",
        None,
        parse.urlencode({"q": place}),
        None,
    )
)
res = request.urlopen(url)
geojson = StringIO(
    json.dumps(
        {
            "type": "FeatureCollection",
            "features": json.loads(res.read().decode()),
        }
    )
)
gdf = gpd.read_file(geojson)

center = (
    gdf.loc[:, "geometry"].y.mean(),
    gdf.loc[:, "geometry"].x.mean(),
)
m = folium.Map(location=center, zoom_start=7)
gdf.apply(
    lambda s: folium.Marker(
        location=(
            s["geometry"].y,
            s["geometry"].x,
        )
    ).add_to(m),
    axis=1,
)
m